# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Script to create a visualization of the ACE2 receptor

In [ ]:
from bioexplorer import BioExplorer, Protein, Sugars, Quaternion
be = BioExplorer('localhost:5000')
be.reset()
print(be.version())

In [ ]:
# Resources
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
complex_folder = resource_folder + 'pdb/glycans/complex/'

protein_radius_multiplier = 1.0
protein_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS

In [ ]:
# Camera position
status = be.core_api().set_camera(
    orientation=[0.707, 0.0, -0.707, 0.0],
    position=[-25.772, 0.0, 0.005],
    target=[0.0, 0.0, 0.005]
)

In [ ]:
# Protein
ace2_receptor = Protein(sources=[pdb_folder + '6m18.pdb'],
                        load_hydrogen=False,
                        load_non_polymer_chemicals=False)
name = be.NAME_RECEPTOR
status = be.add_protein(
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    name=name, protein=ace2_receptor
)

## Protein information (AA sequences and glycosylation sites)

In [ ]:
be.get_protein_amino_acid_information(assembly_name=name, name=name)

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=2)

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Set1', palette_size=5)

## Visualization of amino acids sequence

In [ ]:
status = be.set_protein_amino_acid_sequence_as_ranges(
    assembly_name=name, name=name, amino_acid_ranges=[100,150])
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of glycosylation sites

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Visualization of functional regions

In [ ]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_green = [0.0, 0.5, 0.0]
light_green = [0.0, 1.0, 0.0]

region_indices_and_colors = [
    [   1, grey       ], [   5, grey],  [  30, light_green],  [  41, dark_green],
    [  82, light_green], [  84, grey],  [ 353, grey       ],  [ 357, grey      ],
    [ 800, grey       ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys', palette_size=5)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2])

## Add glycans to protein

In [ ]:
glycan_name = name + '_Complex'
glycans = Sugars(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    assembly_name=name, name=glycan_name, protein_name=name,
    source=complex_folder + '36.pdb',
    representation=be.REPRESENTATION_ATOMS_AND_STICKS,
)
status = be.add_glycans(glycans)

## Materials

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

In [ ]:
status = be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    user_parameter=1, specular_exponent=50)

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5
params.shadows = 1.0
params.soft_shadows = 1.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshot

In [ ]:
from mediamaker import MovieMaker
mm = MovieMaker(be)

In [ ]:
mm.create_snapshot(
    path='/tmp/ACE2_receptor.png',
    size=[512,512], samples_per_pixel=64)